In [1]:
!pip install jupyter-dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.6 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3.7


In [2]:
import dash
app = dash.Dash(__name__)



In [3]:
import requests

# URL of the dataset
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

# Send a GET request
response = requests.get(url)

# Ensure we got a valid response
if response.status_code == 200:
    with open('spacex_launch_dash.csv', 'wb') as file:
        file.write(response.content)
    print("response status code :",response.status_code )
else:
    print("Failed to fetch the dataset.")

response status code : 200


In [4]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

options=[{"label": "All", "value": "All"}]
options.extend([{'label': value, 'value': value} for value in spacex_df['Launch Site'].unique()])

marks = {
    1000: '1k',
    2000: '2k',
    3000: '3k',
    4000: '4k',
    5000: '5k',
    6000: '6k',
    7000: '7k',
    8000: '8k',
    9000: '9k',
    10000: '10k'
}


# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=options,
                                            value="All",
                                            placeholder="Select a Launch Site here",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)

                                dcc.RangeSlider(id = 'payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks=marks,
                                                value=[min_payload , max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'), Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):

  if entered_site == 'All':
    filtered_df = spacex_df[spacex_df['class'] == 1]
    pie_data= filtered_df['Launch Site'].value_counts()



    return px.pie(labels=pie_data.index.to_list(),values= pie_data.values.tolist(),names=pie_data.index.to_list(),title='Overall success Rate by Site Pie Chart')

  else:
    # return the outcomes piechart for a selected site
    #print("else case to be developped")
    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    pie_data= filtered_df['class'].value_counts()
    print(pie_data)

    return px.pie(labels=pie_data.index.to_list(),values= pie_data.values.tolist(),names=pie_data.index.to_list(),title= entered_site +'  Success Rate Pie Chart',color_discrete_sequence=["red", "green"])

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site,selected_range):

  if entered_site == 'All':

    return px.scatter(x=spacex_df['Payload Mass (kg)'], y=spacex_df['class'], title='Payload Mass vs Class Sample Scatter Plot for all Sites', color=spacex_df['Booster Version Category'])

  else:

    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= selected_range[0]) & (filtered_df['Payload Mass (kg)'] <= selected_range[1])]

    return px.scatter(x=filtered_df['Payload Mass (kg)'], y=filtered_df['class'], title='Payload Mass vs Class Scatter Plot for '+ entered_site , color=filtered_df['Booster Version Category'])

# Run the app
if __name__ == '__main__':
    print("app should run")
    app.run_server(mode='external')


<ipython-input-4-ce2a587e17e3>:4: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`

<ipython-input-4-ce2a587e17e3>:5: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`



app should run


<IPython.core.display.Javascript object>